In [1]:
using FFTW
using Zygote
using FiniteDifferences
using ChainRulesCore
using Base.Threads
using BenchmarkTools

n=4
plan = plan_rfft(rand(n,n),flags=FFTW.MEASURE)
function test(x,y,p)
    sum( sin.(p \ ((p*x) .* (p*y))).^3 )
end
x = rand(n,n)
#x .= x.+conj.(x)
y = rand(n,n)
#y .= y.+conj.(y)
display(test(x,y,plan))
tmp = a->test(a,y,plan)
gdat = Zygote.gradient(tmp,x)[1]
Δ = grad(central_fdm(3,1), tmp, x)[1]
display(gdat)
display(Δ)
display(maximum(abs.(vec(Δ) .- vec(gdat))[1:end-1])/mean(abs.(vec(Δ))))

-14.228329446946747

4×4 Matrix{Float64}:
 3.24888  3.82009  2.20408  2.91033
 2.23594  3.21013  3.644    3.67196
 3.03525  2.78591  3.92903  2.86522
 3.36648  2.11777  3.77078  4.14255

4×4 Matrix{Float64}:
 3.24888  3.82009  2.20408  2.91033
 2.23594  3.21013  3.644    3.67196
 3.03525  2.78591  3.92903  2.86522
 3.36648  2.11777  3.77078  4.14255

2.2307453332893926e-10

In [65]:
using PaddedViews
function vGTGu(g,x,y,p,nout)
    Gu = ifftshift( p \ ((p * g) .* (p * x)) )[1:nout]
    Gv = ifftshift( p \ ((p * g) .* (p * y)) )[1:nout]
    sum(Gv .* Gu)
end

g = rand(16)
u = rand(4)
v = rand(4)

nout = size(g)[1] - size(u)[1]
x = sym_paddedviews(0,u,g)[1]
y = sym_paddedviews(0,v,g)[1]
plan = plan_rfft(g,flags=FFTW.MEASURE)

tmp = a -> vGTGu(a,x,y,plan,nout)
gdat = Zygote.gradient(tmp,g)[1]
Δ = grad(central_fdm(3,1), tmp,g)[1]
display(maximum(abs.(vec(Δ) .- vec(gdat))[1:end-1])/mean(abs.(vec(Δ))))
    

1.4730969829661945e-12

In [54]:
a = [zeros((1,2)),[1 2],zeros((1,2))]

3-element Vector{Matrix{Float64}}:
 [0.0 0.0]
 [1.0 2.0]
 [0.0 0.0]

In [150]:
function v∂GTGu(g,u,v,p)

    nout = size(g)[1] - size(u)[1]
    x = sym_paddedviews(0,u,g)[1]
    y = sym_paddedviews(0,v,g)[1]

    function xGTGy(h)
        Gx = ifftshift( p \ ((p * h) .* (p * x)) )[1:nout,1:nout]
        Gy = ifftshift( p \ ((p * h) .* (p * y)) )[1:nout,1:nout]
        sum(Gx .* Gy)
    end

    (xGTGy(g), Zygote.gradient(xGTGy,g)[1])

end

n1 = 10
n2 = 6
g = rand(n1,n1)
u = rand(n2,n2)
v = rand(n2,n2)
p = plan_rfft(g,flags=FFTW.MEASURE)

gdat = v∂GTGu(g,u,v,p)[2]
tmp = a -> v∂GTGu(a,u,v,p)[1]
Δ = grad(central_fdm(3,1), tmp,g)[1]
display(maximum(abs.(vec(Δ) .- vec(gdat))[1:end-1])/mean(abs.(vec(Δ))))

function v∂GTGu_manual(g,u,v,p)

    n1 = size(g)[1]
    n2 = size(u)[1]
    nout = n1 - n2
    x = sym_paddedviews(0,u,g)[1]
    y = sym_paddedviews(0,v,g)[1]

    Gx = ifftshift( p \ ((p * g) .* (p * x)) )[1:nout,1:nout]
    Gy = ifftshift( p \ ((p * g) .* (p * y)) )[1:nout,1:nout]

    xT = sym_paddedviews(0,reverse(u),g)[1]
    yT = sym_paddedviews(0,reverse(v),g)[1]
    Gx = sym_paddedviews(0,reverse(Gx),g)[1]
    Gy = sym_paddedviews(0,reverse(Gy),g)[1]

    ret = ( p \ (conj.(p * Gy) .* (p * xT)) ) .+
          ( p \ (conj.(p * Gx) .* (p * yT)) )

    if n1 > 2*n2
        return circshift(ret,(-1,-1))
    else
        return circshift(ret,((n1-n2)÷2,(n1-n2)÷2))
    end

end
gdat2 = v∂GTGu_manual(g,u,v,p)
display(gdat)
display(gdat2)
maximum(abs.(gdat-gdat2))

2.3303991414094447e-11

10×10 Matrix{Float64}:
  6.02058      11.5956       19.4282       …  13.9919       -4.33681e-16
  7.48283      20.2727       40.5638          23.0092        2.93033e-15
 15.849        36.9233       62.1111          29.4595        7.64032e-15
 24.6664       55.481        85.2011          38.7185        1.00737e-14
 23.1683       49.6693       74.6281          32.8472        1.0121e-14
 27.6493       46.9813       67.0266       …  24.1132        1.02036e-14
 19.1048       30.3336       44.3004          18.0478        5.9409e-15
 10.0216       12.8555       20.8383           9.60116       1.11405e-15
  4.91973       6.65406      13.5845           2.61007      -1.01205e-15
  2.88658e-15   9.76996e-15   8.88178e-15     -2.66454e-15  -2.16926e-15

10×10 Matrix{Float64}:
 20.2727       40.5638        67.1112       …  1.47872e-14   7.48283
 36.9233       62.1111       104.768           1.22998e-14  15.849
 55.481        85.2011       140.462           1.12088e-14  24.6664
 49.6693       74.6281       123.254           7.45797e-15  23.1683
 46.9813       67.0266       109.847           7.9553e-15   27.6493
 30.3336       44.3004        72.8277       …  8.50805e-15  19.1048
 12.8555       20.8383        36.7023          8.92398e-15  10.0216
  6.65406      13.5845        20.0872          1.42997e-14   4.91973
  1.10845e-14   9.09495e-15    9.09495e-15     1.40569e-14   1.08002e-14
 11.5956       19.4282        33.4496          1.41891e-14   6.02058

78.35088543121525

In [242]:
function v∂GTGu(g,u,v,p)

    nout = size(g)[1] - size(u)[1]
    x = sym_paddedviews(0,u,g)[1]
    y = sym_paddedviews(0,v,g)[1]

    function xGTGy(h)
        Gx = ifftshift( p \ ((p * h) .* (p * x)) )[1:nout]
        Gy = ifftshift( p \ ((p * h) .* (p * y)) )[1:nout]
        sum(Gx .* Gy)
    end

    (xGTGy(g), Zygote.gradient(xGTGy,g)[1])

end



function v∂GTGu_manual(g,u,v,p,m)

    n1 = size(g)[1]
    n2 = size(u)[1]
    nout = n1 - n2
    x = sym_paddedviews(0,u,g)[1]
    y = sym_paddedviews(0,v,g)[1]

    Gx = ifftshift( p \ ((p * g) .* (p * x)) )[1:nout]
    Gy = ifftshift( p \ ((p * g) .* (p * y)) )[1:nout]

    xT = sym_paddedviews(0,reverse(u),g)[1]
    yT = sym_paddedviews(0,reverse(v),g)[1]
    Gx = sym_paddedviews(0,reverse(Gx),g)[1]
    Gy = sym_paddedviews(0,reverse(Gy),g)[1]

    ret = ( p \ (conj.(p * Gy) .* (p * xT)) ) .+
          ( p \ (conj.(p * Gx) .* (p * yT)) )

    circshift(ret,m)

end

n1,n2 = 16,14
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,-4)
display([round.(gdat),round.(gdat2)])

n1,n2 = 16,12
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,0)
display([round.(gdat),round.(gdat2)])

n1,n2 = 16,10
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,4)
display([round.(gdat),round.(gdat2)])

n1,n2 = 16,8
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = ifftshift(v∂GTGu_manual(g,u,v,p,0))
display([round.(gdat),round.(gdat2)])

n1,n2 = 16,6
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,-4)
display([round.(gdat),round.(gdat2)])

n1,n2 = 16,4
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,0)
display([round.(gdat),round.(gdat2)])

n1,n2 = 16,2
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,4)
display([round.(gdat),round.(gdat2)])

n1,n2 = 10,8
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,-1)
display([round.(gdat),round.(gdat2)])

n1,n2 = 10,6
g,u,v = rand(n1), rand(n2), rand(n2)
p = plan_rfft(g,flags=FFTW.MEASURE)
gdat = v∂GTGu(g,u,v,p)[2]
gdat2 = v∂GTGu_manual(g,u,v,p,3)
display([round.(gdat),round.(gdat2)])

2-element Vector{Vector{Float64}}:
 [6.0, 6.0, 8.0, 7.0, 4.0, 6.0, 9.0, 7.0, 7.0, 6.0, -0.0, 6.0, 8.0, 7.0, 7.0, 6.0]
 [6.0, 6.0, 8.0, 7.0, 4.0, 6.0, 9.0, 7.0, 7.0, 6.0, -0.0, 6.0, 8.0, 7.0, 7.0, 6.0]

2-element Vector{Vector{Float64}}:
 [7.0, 8.0, 10.0, 9.0, 9.0, 7.0, 7.0, 8.0, 7.0, 5.0, 5.0, 1.0, -0.0, 1.0, 2.0, 5.0]
 [7.0, 8.0, 10.0, 9.0, 9.0, 7.0, 7.0, 8.0, 7.0, 5.0, 5.0, 1.0, -0.0, 1.0, 2.0, 5.0]

2-element Vector{Vector{Float64}}:
 [6.0, 10.0, 10.0, 16.0, 20.0, 19.0, 18.0, 16.0, 19.0, 13.0, 9.0, 7.0, 5.0, 3.0, -0.0, 3.0]
 [6.0, 10.0, 10.0, 16.0, 20.0, 19.0, 18.0, 16.0, 19.0, 13.0, 9.0, 7.0, 5.0, 3.0, 0.0, 3.0]

2-element Vector{Vector{Float64}}:
 [0.0, 1.0, 2.0, 3.0, 4.0, 6.0, 6.0, 7.0, 8.0, 7.0, 5.0, 5.0, 4.0, 2.0, 2.0, 1.0]
 [-0.0, 1.0, 2.0, 3.0, 4.0, 6.0, 6.0, 7.0, 8.0, 7.0, 5.0, 5.0, 4.0, 2.0, 2.0, 1.0]

2-element Vector{Vector{Float64}}:
 [4.0, 2.0, -0.0, 1.0, 1.0, 4.0, 6.0, 7.0, 10.0, 9.0, 10.0, 10.0, 10.0, 9.0, 8.0, 6.0]
 [4.0, 2.0, -0.0, 1.0, 1.0, 4.0, 6.0, 7.0, 10.0, 9.0, 10.0, 10.0, 10.0, 9.0, 8.0, 6.0]

2-element Vector{Vector{Float64}}:
 [4.0, 3.0, 2.0, 2.0, -0.0, 2.0, 4.0, 5.0, 6.0, 6.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0]
 [4.0, 3.0, 2.0, 2.0, 0.0, 2.0, 4.0, 5.0, 6.0, 6.0, 5.0, 4.0, 3.0, 3.0, 3.0, 4.0]

2-element Vector{Vector{Float64}}:
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

2-element Vector{Vector{Float64}}:
 [3.0, 4.0, 3.0, 1.0, 2.0, 3.0, 1.0, 0.0, 1.0, 3.0]
 [3.0, 4.0, 3.0, 1.0, 2.0, 3.0, 1.0, 0.0, 1.0, 3.0]

2-element Vector{Vector{Float64}}:
 [1.0, 1.0, 2.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 0.0]
 [1.0, 1.0, 2.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 0.0]